# Calculate lower-tropospheric stability

LTS defined as difference of potential temperature between 3 km and the surface following Li et al, JGR, 2014, doi: 10.1002/2013JD020669.

Code written by Elzina Bala and Aiko Voigt.

In [1]:
import xarray as xr
import dask
import shutil

In [2]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:37282 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 48 Memory: 134.22 GB


Load dictionary will all simulations

In [3]:
import sys
sys.path.append('/pf/b/b380459/nawdex-hackathon/shared')
import dict_nawdexsims

simdict   = dict_nawdexsims.simdictionary()

In [4]:
# constants declaration, values taken from ICON source code
Cp = 1004.64    # cpd in ICON, [J/K/kg] specific heat at constant pressure
Ra = 287.04_wp  # rd in ICON, [J/K/kg] gas constant

In [5]:
def load_data(expid):
    
    path  = '/scratch/b/b380459/icon_4_hackathon/'
    fname = path+'/'+expid+'/'+expid+'_2016*_fg_DOM01_ML_*.nc'
    ds = ( xr.open_mfdataset(fname, combine='by_coords',parallel=True, 
                             engine='h5netcdf', chunks={'time': 1})
           [['temp', 'pres']].rename({'ncells_2': 'ncells'}) )
    
    return ds

Note: Level with index 52 is at 3km height above ocean, level with index 74 is the surface. We use a fixed height level here because we intend to only look at the ocean points later.

In [ ]:
for sim in list(simdict.keys()):
    print('Working on:', sim)
    ds = load_data(sim)
    ds = ds.isel(height=[52,74])
    ds['theta'] = ds['temp'] * (100000./ds['pres'])**(Ra/Cp)
    ds['lts'] = (ds['theta'].isel(height=1) - ds['theta'].isel(height=0) ) / 3000.
    ds = ds.drop(['temp','pres','theta'])
    ds.attrs['units'] = 'Kelvin m-1'
    ds.attrs['description'] = 'lower tropospheric stability according to Li et al., JGR, 2014, doi: 10.1002/2013JD020669'
    ds.attrs['simulation'] = sim
    # store to zarr store
    zarr_store = '/scratch/b/b380459/icon_4_hackathon/lower_tropo_stability/lts_'+sim+'.zarr'
    # remove any zarr_store with same name that might have been created previously
    shutil.rmtree(zarr_store, ignore_errors=True)
    ds.to_zarr(zarr_store)

Working on: nawdexnwp-80km-mis-0001
Working on: nawdexnwp-80km-mis-0002
Working on: nawdexnwp-80km-mis-0003
Working on: nawdexnwp-80km-mis-0004
Working on: nawdexnwp-80km-mis-0005
Working on: nawdexnwp-80km-mis-0006
Working on: nawdexnwp-80km-mis-0007
Working on: nawdexnwp-80km-mis-0008
Working on: nawdexnwp-80km-mis-0009
Working on: nawdexnwp-80km-mis-0010
Working on: nawdexnwp-40km-mis-0001
Working on: nawdexnwp-40km-mis-0002
Working on: nawdexnwp-40km-mis-0003
Working on: nawdexnwp-40km-mis-0004
Working on: nawdexnwp-40km-mis-0005
Working on: nawdexnwp-40km-mis-0006
Working on: nawdexnwp-40km-mis-0007
Working on: nawdexnwp-40km-mis-0008
Working on: nawdexnwp-40km-mis-0009


Clean up

In [ ]:
client.close()